### 量化工具 llama-quantize
量化是 llama.cpp 最受歡迎的功能，llama.cpp 支援的量化方法相當廣泛，從 8-Bit、6-Bit 甚至到 2-Bit、1-Bit 都有，同時也支援混精度、線性、非線性等量化方法。

#### 1. 基本的量化方法只要使用 `llama-quantize` 這個工具便可以進行操作：
* 以下指令會將模型量化為 4-Bit 格式，這裡的 `Q4_K_M` 代表「以 `Q4_K` 為基本型態的 Medium 大小」，意思是大部分的模型權重都是 4-Bit 的 `Q4_K` 型態，但是有一部分的權重會用 6-Bit 的 `Q6_K` 型態，因此最後的**平均權重位元 (Bits Per Weight, BPW)** 其實會將近 5.0。
    * 另外也有 `Q4_K_S` 是混合 `Q4K` 與 `Q5K` 的格式，其 BPW 會更貼近 4.0 一些。
* 透過 `./build/bin/llama-quantize --help` 可以看到，還有許多 `IQ` 開頭的量化方法。
    * 例如極低位元數的 `IQ2_XXS` 與 `IQ1_S` 等。

參數說明

In [ ]:
%%bash

cd llama.cpp

./build/bin/llama-quantize --help

將模型量化為 4-Bit 格式 (`Q4_K_M`)

In [ ]:
%%bash

cd llama.cpp

./build/bin/llama-quantize \
    ./llama-3-8b-inst.gguf \
    ./llama-3-8b-inst.Q4_K_M.gguf Q4_K_M

#### 2. 透過另外一個程式 `llama-imatrix` 的協助來使用這些量化方法。
* `llama-imatrix` 指的是**重要性矩陣 (Importance Matrix)** ，這個程式會透過一份校準資料集來評估哪些模型權重是重要的，這些重要的權重需要保護起來，才能在極低位元的量化下維持模型的表現。

安裝套件
* 這個模組是 Hugging Face 的 datasets 庫，常用於載入和處理各種資料集，特別是在 NLP 任務中。

In [2]:
%%bash
pip install datasets

  Using cached dill-0.3.8-py3-none-any.whl.metadata (10 kB)
  Using cached pandas-2.2.3-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (89 kB)
  Using cached xxhash-3.5.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (12 kB)
  Using cached multiprocess-0.70.16-py311-none-any.whl.metadata (7.2 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.2-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached frozenlist-1.6.0-cp311-cp311-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (16 kB)
  Using cached multidict-6.4.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.3 kB)
  Using cached propcache-0.3.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (10 kB)
  Using cached yarl-1.20.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metada

需要先蒐集一份純文字資料：

In [4]:
from datasets import load_dataset

ds_path = "bigscience-data/roots_zh-tw_wikipedia"
ds = load_dataset(ds_path, split="train", streaming=True)
text = [item["text"] for _, item in zip(range(128), ds)]
text = "\n".join(text)
with open("zh-wiki.txt", "wt", encoding="UTF-8") as fp:
    fp.write(text)

接著丟入 `llama-imatrix` 裡面：
* 大約兩三分鐘的時間就能跑完，並產生一份 `imatrix.dat` 檔案。

In [ ]:
%%bash

cd llama.cpp

./build/bin/llama-imatrix \
    -ngl 99 -c 8192 -f ../zh-wiki.txt \
    -m llama-3-8b-inst.gguf

build: 1 (a3c3084) with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu


llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from llama-3-8b-inst.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = 5f0b02c75b57c5855da9ae460ce51323ea669d8a
llama_model_loader: - kv   3:                           general.finetune str              = 5f0b02c75b57c5855da9ae460ce51323ea669d8a
llama_model_loader: - kv   4:                         general.size_label str              = 8.0B
llama_model_loader: - kv   5:                          llama.block_count u32              = 32
llama_model_loader: - kv   6:                       llama.context_length u32              = 8192
llama_model_loader: -

1 hours 50.73 minutes
[1]15.2444,[2]12.2086,[3]11.5078,[4]11.8649,[5]13.2535,[6]14.3845,[7]14.3393,[8]13.8441,[9]13.3747,[10]13.6807,

#### 3. 將這份檔案用在量化工具上，便能產生極低位元的量化模型了：
* 以 Breeze-7B 實測，原本 FP16 為 14 GB，量化成 `Q4_KM` 只剩下 4.3 GB，到了 `IQ2_XXS` 只剩下 2 GB，最極端的 `IQ1_S` 更是只剩下 1.7 GB！但 7B 的模型量化到這個程度，其實已經接近無法使用的狀態了。
* 根據筆者的經驗，8-Bit 和 6-Bit 幾乎不會有損失的感覺，到了 4-Bit 以下才會有明顯感受到不同，而參數量越大的機型，受到量化的影響則越小。
    * 不過這是個人的主觀感受，最後選擇使用多少位元的量化，還是要根據實際應用的狀況進行過充分的評測後再來決定。

In [1]:
%%bash

cd llama.cpp

./build/bin/llama-quantize \
    --imatrix imatrix.dat \
    llama-3-8b-inst.gguf \
    llama-3-8b-inst.IQ2_XXS.gguf IQ2_XXS

main: build = 1 (a3c3084)
main: built with cc (Ubuntu 11.4.0-1ubuntu1~22.04) 11.4.0 for x86_64-linux-gnu
main: quantizing 'llama-3-8b-inst.gguf' to 'llama-3-8b-inst.IQ2_XXS.gguf' as IQ2_XXS
llama_model_loader: loaded meta data with 27 key-value pairs and 291 tensors from llama-3-8b-inst.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.type str              = model
llama_model_loader: - kv   2:                               general.name str              = 5f0b02c75b57c5855da9ae460ce51323ea669d8a
llama_model_loader: - kv   3:                           general.finetune str              = 5f0b02c75b57c5855da9ae460ce51323ea669d8a
llama_model_loader: - kv   4:                         general.size_label str              = 8.0B
llama_model_loader: - k

load_imatrix: imatrix dataset='../zh-wiki.txt'
load_imatrix: loaded 224 importance matrix entries from imatrix.dat computed on 320 chunks
prepare_imatrix: have 224 importance matrix entries


size =  1002.00 MiB ->   344.44 MiB
[   2/ 291]                   output_norm.weight - [ 4096,     1,     1,     1], type =    f32, size =    0.016 MB
[   3/ 291]                    token_embd.weight - [ 4096, 128256,     1,     1], type =    f16, 
====== llama_model_quantize_impl: did not find weights for token_embd.weight
converting to q2_K .. size =  1002.00 MiB ->   164.39 MiB
[   4/ 291]                  blk.0.attn_k.weight - [ 4096,  1024,     1,     1], type =    f16, converting to iq2_xxs .. size =     8.00 MiB ->     1.03 MiB
[   5/ 291]               blk.0.attn_norm.weight - [ 4096,     1,     1,     1], type =    f32, size =    0.016 MB
[   6/ 291]             blk.0.attn_output.weight - [ 4096,  4096,     1,     1], type =    f16, converting to iq2_xxs .. size =    32.00 MiB ->     4.12 MiB
[   7/ 291]                  blk.0.attn_q.weight - [ 4096,  4096,     1,     1], type =    f16, converting to iq2_xxs .. size =    32.00 MiB ->     4.12 MiB
[   8/ 291]                  b


main: quantize time = 115021.70 ms
main:    total time = 115021.70 ms
